In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
from os import listdir
from os.path import join
import os, json, csv

##Combined File

In [6]:
def create_nvd_dict(year):
    filename = join("/content/drive/MyDrive/Python_Training/Final_Project_ModuleB/jsonFile/nvdcve-1.1-" + str(year) + ".json")
    print("Opening: " + filename)
    with open(filename, encoding="utf8") as json_file:
        cve_dict = json.load(json_file)
    return(cve_dict)

def generate_csv():
    columns = ['cve_id', 'report', 'exploitabilityScore', 'impactScore', 'baseScore']
    list = listdir("/content/drive/MyDrive/Python_Training/Final_Project_ModuleB/jsonFile/")
    number_files = len(list)
    for year in range(2002,2002 + number_files):
        year_in_string = str(year)
        cve_dict = create_nvd_dict(year)
        fileName = 'NVD_'+ year_in_string + '_threat_train.csv'
        folder_path = '/content/drive/MyDrive/Python_Training/ModuleA_Project/nvd'
        file_path = os.path.join(folder_path, fileName)
        with open('/content/drive/MyDrive/Python_Training/ModuleA_Project/nvd/' + fileName, 'w', newline='') as f_output:
            print("Creating: " + fileName)
            csv_output = csv.DictWriter(f_output, fieldnames=columns)
            csv_output.writeheader()
            for item in cve_dict['CVE_Items']:
                cve_id = item['cve']['CVE_data_meta']['ID']
                report = item['cve']['description']['description_data'][0]['value']

                if "REJECT" not in report:
                    baseScore = item.get('impact', {}).get('baseMetricV2', {}).get('cvssV2', {}).get('baseScore')
                    exploitabilityScore = item.get('impact', {}).get('baseMetricV2', {}).get('exploitabilityScore')
                    impactScore = item.get('impact', {}).get('baseMetricV2', {}).get('impactScore')
                    if baseScore is not None:
                        data_row = {'cve_id': cve_id, 'report': report, 'exploitabilityScore': exploitabilityScore, 'impactScore': impactScore, 'baseScore': baseScore}
                        csv_output.writerow(data_row)

In [7]:
generate_csv()

Opening: /content/drive/MyDrive/Python_Training/Final_Project_ModuleB/jsonFile/nvdcve-1.1-2002.json
Creating: NVD_2002_threat_train.csv
Opening: /content/drive/MyDrive/Python_Training/Final_Project_ModuleB/jsonFile/nvdcve-1.1-2003.json
Creating: NVD_2003_threat_train.csv
Opening: /content/drive/MyDrive/Python_Training/Final_Project_ModuleB/jsonFile/nvdcve-1.1-2004.json
Creating: NVD_2004_threat_train.csv
Opening: /content/drive/MyDrive/Python_Training/Final_Project_ModuleB/jsonFile/nvdcve-1.1-2005.json
Creating: NVD_2005_threat_train.csv
Opening: /content/drive/MyDrive/Python_Training/Final_Project_ModuleB/jsonFile/nvdcve-1.1-2006.json
Creating: NVD_2006_threat_train.csv
Opening: /content/drive/MyDrive/Python_Training/Final_Project_ModuleB/jsonFile/nvdcve-1.1-2007.json
Creating: NVD_2007_threat_train.csv
Opening: /content/drive/MyDrive/Python_Training/Final_Project_ModuleB/jsonFile/nvdcve-1.1-2008.json
Creating: NVD_2008_threat_train.csv
Opening: /content/drive/MyDrive/Python_Training/

In [8]:
def generate_CombinedFile():
    folder_path = "/content/drive/MyDrive/Python_Training/ModuleA_Project/nvd/"
    file_list = listdir(folder_path)
    number_files = len(file_list)
    dict_of_reports = {}
    dict = []
    for year in range(2002, 2002 + number_files):
        year_in_string = str(year)
        file_name = 'NVD_' + year_in_string + '_threat_train.csv'
        file_path = folder_path + file_name

        dict_of_reports[year_in_string] = pd.read_csv(file_path)
        dict.append(dict_of_reports[year_in_string])

    df = pd.concat(dict, ignore_index=True)
    output_file_path = "/content/drive/MyDrive/Python_Training/ModuleA_Project/combined_data.csv"
    df.to_csv(output_file_path, index=False)

    return df

In [9]:
df = generate_CombinedFile()
df

,cve_id,report,exploitabilityScore,impactScore,baseScore
0,CVE-1999-0001,ip_input.c in BSD-derived TCP/IP implementatio...,10.0,2.9,5.0
1,CVE-1999-0002,Buffer overflow in NFS mountd gives root acces...,10.0,10.0,10.0
2,CVE-1999-0003,Execute commands as root via buffer overflow i...,10.0,10.0,10.0
3,CVE-1999-0004,"MIME buffer overflow in email clients, e.g. So...",10.0,2.9,5.0
4,CVE-1999-0005,Arbitrary command execution via IMAP buffer ov...,10.0,10.0,10.0
...,...,...,...,...,...
180041,CVE-2022-35648,Nautilus treadmills T616 S/N 100672PRO21140001...,3.9,2.9,2.1
180042,CVE-2022-35861,pyenv 1.2.24 through 2.3.2 allows local users ...,3.9,6.4,4.6
180043,CVE-2022-36728,Library Management System v1.0 was discovered ...,10.0,6.4,7.5
180044,CVE-2022-36880,The Read Mail module in Webmin 1.995 and Userm...,8.6,2.9,4.3
